# Day 12 Challenge

In [1]:
import aocd
import numpy as np
import datetime

from numpy import lcm

session = "53616c7465645f5f5d64e0f6b2811f4d18eb862dae5aa906e4d25c0f6a1d5944c89433699f1b1175fdea140e981da4a5"
day     = datetime.datetime.now().day
year    = datetime.datetime.now().year
data    = aocd.get_data(session=session, year=year, day=day)

In [2]:
positions = []
for row in data.split("\n"):
    for field in row.split(", "):
        positions.append(int(field.split("=")[1].replace(">", "")))
        
positions = np.array(positions)

moon_positions = []
for line in positions.reshape(4,3):
    moon_positions.append({"x":line[0], "y": line[1], "z": line[2]})

In [57]:
from functools import reduce
from math import gcd

def lcm2(nums):

    return reduce(lambda n, m: (n * m)//gcd(n, m), nums)

In [59]:
from itertools import permutations, count

class OrbitalSystem:
    def __init__(self, timesteps=10):
        self._timestep = 0
        self._timesteps = timesteps
        self.moons = []
        self._cycle_lengths = {"x": None, "y": None, "z": None}
    
    def add_moon(self, name, position):
        self.moons.append(
            Moon(
                name=name,
                position=position
            )
        )
    
    def print_state(self, step):
        print(f"\n\nstep {step}")
        print("\npos")
        print(self.moons[0]._name, self.moons[0]._position, self.moons[0]._original_position)
        print(self.moons[1]._name, self.moons[1]._position, self.moons[1]._original_position)
        print(self.moons[2]._name, self.moons[2]._position, self.moons[2]._original_position)
        print(self.moons[3]._name, self.moons[3]._position, self.moons[3]._original_position)
        print("\nvel")
        print(self.moons[0]._name, self.moons[0]._velocity, self.moons[0]._original_velocity)
        print(self.moons[1]._name, self.moons[1]._velocity, self.moons[1]._original_velocity)
        print(self.moons[2]._name, self.moons[2]._velocity, self.moons[2]._original_velocity)
        print(self.moons[3]._name, self.moons[3]._velocity, self.moons[3]._original_velocity)
    
    def simulate_movement(self, part2=False):
        pairs = list(permutations(range(len(self.moons)), 2))
        
        for step in range(1, self._timesteps+1):
            
            self._timestep += 1
            for pair in pairs:
                moon1 = self.moons[pair[0]]
                moon2 = self.moons[pair[1]]
                moon1.update_velocity(moon=moon2)

            for moon in self.moons:
                moon.update_position()
                moon.update_energies()
            
            
            if part2:
                self.check_against_initial()

                # break simulation if all three cycle lens have been found
                if self._cycle_lengths["x"] and self._cycle_lengths["y"] and self._cycle_lengths["z"]:
                    print(lcm2(
                        [
                            self._cycle_lengths["x"],
                            self._cycle_lengths["y"],
                            self._cycle_lengths["z"]
                        ]
                    ))
                    break
                
    def check_against_initial(self):
        for axis in ["x", "y", "z"]:
            validation_list = []
            for moon in self.moons:
                validation_list.append(moon.check_against_initial(axis))
            if sum(validation_list) == len(self.moons):
                if not self._cycle_lengths[axis]:
                    self._cycle_lengths[axis] = self._timestep

    def return_total_energy(self):
        total = 0
        for moon in self.moons:
            total += moon._total_energy
        return total
    
class Moon:
    def __init__(self, name, position):
        self._name = name
        self._position = position.copy()
        self._velocity = {"x": 0, "y": 0, "z": 0}
        self._potential_energy = 0
        self._kinetic_energy = 0
        self._total_energy = 0
        
        self._original_position = position.copy()
        self._original_velocity = {"x": 0, "y": 0, "z": 0}
        
    def check_against_initial(self, axis):
        if self._position[axis] == self._original_position[axis]:
            if self._velocity[axis] == self._original_velocity[axis]:
                return 1
            else:
                return 0
        else:
            return 0
        
    def update_velocity(self, moon):
        cp = moon._position
        op = self._position
        
        for i in ["x", "y", "z"]:
            if cp[i] > op[i]:
                self._velocity[i] = self._velocity[i] + 1
            elif cp[i] < op[i]:
                self._velocity[i] = self._velocity[i] - 1
            elif cp[i] == op[i]:
                self._velocity[i] = self._velocity[i]

    def update_energies(self):
        self._potential_energy = abs(self._position["x"]) + abs(self._position["y"]) + abs(self._position["z"])
        self._kinetic_energy = abs(self._velocity["x"]) + abs(self._velocity["y"]) + abs(self._velocity["z"])
        self._total_energy = self._kinetic_energy * self._potential_energy
    
    def update_position(self):
        for axis in ["x", "y", "z"]:
            self._position[axis] += self._velocity[axis]

## Testing

In [60]:
# Example 1 (2772)

jupyter_system = OrbitalSystem(timesteps=500000)
jupyter_system.add_moon(name="Io", position={"x":-1, "y":0, "z":2})
jupyter_system.add_moon(name="Europa", position={"x":2, "y":-10, "z":-7})
jupyter_system.add_moon(name="Ganymede", position={"x":4, "y":-8, "z":8})
jupyter_system.add_moon(name="Callisto", position={"x":3, "y":5, "z":-1})
jupyter_system.simulate_movement(part2=True)

2772


In [61]:
# Example 2: 4686774924

jupyter_system = OrbitalSystem(timesteps=500000)
jupyter_system.add_moon(name="Io", position={"x":-8, "y":-10, "z":0})
jupyter_system.add_moon(name="Europa", position={"x":5, "y":5, "z":10})
jupyter_system.add_moon(name="Ganymede", position={"x":2, "y":-7, "z":3})
jupyter_system.add_moon(name="Callisto", position={"x":9, "y":-8, "z":-3})
jupyter_system.simulate_movement(part2=True)

4686774924


## Part 1

In [62]:
jupyter_system = OrbitalSystem(timesteps=1000)
jupyter_system.add_moon(name="Io", position=moon_positions[0])
jupyter_system.add_moon(name="Europa", position=moon_positions[1])
jupyter_system.add_moon(name="Ganymede", position=moon_positions[2])
jupyter_system.add_moon(name="Callisto", position=moon_positions[3])
jupyter_system.simulate_movement(part2=False)
jupyter_system.return_total_energy()

7988

## Part 2

In [63]:
jupyter_system = OrbitalSystem(timesteps=500000)
jupyter_system.add_moon(name="Io", position=moon_positions[0])
jupyter_system.add_moon(name="Europa", position=moon_positions[1])
jupyter_system.add_moon(name="Ganymede", position=moon_positions[2])
jupyter_system.add_moon(name="Callisto", position=moon_positions[3])
jupyter_system.simulate_movement(part2=True)

337721412394184
